# IBIT Backtest: AMMA vs Trend + Mining + OU

This notebook now uses the **exact combination model logic from Research 02 FINAL CELL D pipeline**:
- Build `trend_mining_pos = 0.5*Trend + 0.5*Mining`
- Sweep OU blend `a` and keep only improvements in Sharpe
- Set `best_combo_pos = best['pos']`
- Apply that BTC-selected position to IBIT via date alignment/ffill


In [ ]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = Path.cwd().resolve()
crypto_root = None
for c in [cwd, cwd / 'Crypto', cwd.parent, cwd.parent / 'Crypto', cwd.parent.parent / 'Crypto']:
    if (c / 'Data' / 'IBIT_Data.csv').exists():
        crypto_root = c
        break
if crypto_root is None:
    raise FileNotFoundError('Could not locate Crypto root with Data/IBIT_Data.csv')

quant_root = crypto_root.parent
if str(crypto_root) not in sys.path:
    sys.path.insert(0, str(crypto_root))

import config as cfg
from Backtest.amma import amma_from_ibit_csv
from Data.raw_data_loader import load_raw_crypto_csv
from Models.trend import trend_signal
from Models.mining import mining_signal
from Models.ou import ou_signal


In [ ]:
# Metrics/backtest helpers

def run_backtest_from_position(price: pd.Series, pos: pd.Series, fee_bps: float, slippage_bps: float) -> dict:
    price = price.astype(float)
    ret = price.pct_change().fillna(0.0)
    pos = pos.reindex(price.index).fillna(0.0)
    turn = pos.diff().abs().fillna(0.0)
    cost = (fee_bps + slippage_bps) / 1e4 * turn
    net_ret = (pos * ret) - cost
    net_equity = (1.0 + net_ret).cumprod()
    return {"net_ret": net_ret, "net_equity": net_equity, "turnover": turn}


def sr(x: pd.Series) -> float:
    x = x.dropna()
    if x.std(ddof=1) == 0 or len(x) < 10:
        return np.nan
    return float(np.sqrt(cfg.DAYS_PER_YEAR) * x.mean() / x.std(ddof=1))


def annual_vol(x: pd.Series) -> float:
    x = x.dropna()
    return float(np.sqrt(252) * x.std(ddof=1)) if len(x) > 1 else np.nan


def cagr(x: pd.Series) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    years = len(eq) / 252.0
    return float(eq.iloc[-1] ** (1.0 / years) - 1.0) if years > 0 else np.nan


def sortino(x: pd.Series) -> float:
    x = x.dropna()
    dn = x[x < 0]
    if len(dn) < 2:
        return np.nan
    dv = dn.std(ddof=1) * np.sqrt(252)
    return float((x.mean() * 252) / dv) if dv > 0 else np.nan


def maxdd(x: pd.Series) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    dd = eq / eq.cummax() - 1.0
    return float(dd.min())


def winrate(x: pd.Series) -> float:
    x = x.dropna()
    return float((x > 0).mean()) if len(x) else np.nan


def metrics_table(ret_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for c in ret_df.columns:
        r = ret_df[c]
        rows.append({
            'Strategy': c,
            'CAGR': cagr(r),
            'Sharpe': sr(r),
            'Sortino': sortino(r),
            'MaxDD': maxdd(r),
            'Volatility': annual_vol(r),
            'WinRate': winrate(r),
        })
    return pd.DataFrame(rows).set_index('Strategy').sort_values('Sharpe', ascending=False)


In [ ]:
# 1) Build exact Research-02 combination model on BTC (FINAL CELL A/B/C logic)

df = load_raw_crypto_csv(cfg.DATA_PATH, start_date=cfg.DATA_START_DATE)
price_btc = df[cfg.PRICE_COLUMN_BTC].astype(float)

trend_pos = trend_signal(
    df,
    price_column=cfg.PRICE_COLUMN_BTC,
    fast_window=cfg.TREND_FAST_WINDOW,
    slow_window=cfg.TREND_SLOW_WINDOW,
    long_only=True,
    leverage_aggressive=1.0,
    leverage_neutral=0.5,
    leverage_defensive=0.0,
).reindex(price_btc.index).fillna(0.0)

mining_pos = mining_signal(
    df,
    z_window=cfg.MINING_Z_WINDOW,
    entry_z=cfg.MINING_ENTRY_Z,
    exit_z=cfg.MINING_EXIT_Z,
    use_log_edge=cfg.MINING_USE_LOG_EDGE,
).reindex(price_btc.index).fillna(0.0)

ou_pos = ou_signal(
    price_btc,
    window=cfg.OU_WINDOW,
    entry_z=cfg.OU_ENTRY_Z,
    exit_z=max(cfg.OU_EXIT_Z, 0.3),
    long_short=False,
).reindex(price_btc.index).fillna(0.0)

# FINAL CELL A exact pattern
trend_mining_pos = (0.5 * trend_pos + 0.5 * mining_pos).clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
trend_mining_pos = trend_mining_pos.shift(1).fillna(0.0)
trend_mining_results = run_backtest_from_position(price_btc, trend_mining_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

# FINAL CELL C-search exact pattern
base_sr = sr(trend_mining_results['net_ret'])
alphas = np.linspace(0.0, 1.0, 21)

best = {
    'alpha_ou': 0.0,
    'sr': base_sr,
    'pos': trend_mining_pos,
    'results': trend_mining_results,
    'label': 'Trend+Mining (0.5/0.5)'
}

ou_pos_aligned = ou_pos.reindex(price_btc.index).fillna(0.0)
for a in alphas:
    combo_pos = ((1 - a) * trend_mining_pos + a * ou_pos_aligned).clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
    combo_pos = combo_pos.shift(1).fillna(0.0)

    combo_results = run_backtest_from_position(price_btc, combo_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
    combo_sr = sr(combo_results['net_ret'])

    if np.isfinite(combo_sr) and combo_sr > best['sr'] + 1e-6:
        best = {
            'alpha_ou': float(a),
            'sr': float(combo_sr),
            'pos': combo_pos,
            'results': combo_results,
            'label': f'Trend+Mining+OU (best a={a:.2f})'
        }

best_combo_pos = best['pos']
best_combo_results = best['results']
print('Selected model from exact Research 02 logic:', best['label'])
print('Selected alpha_ou:', best['alpha_ou'], 'Sharpe:', best['sr'])


In [ ]:
# 2) Apply exact selected combo model to IBIT and compare with AMMA + Buy&Hold

ibit = pd.read_csv(crypto_root / 'Data' / 'IBIT_Data.csv', encoding='utf-8-sig')
ibit['Date'] = pd.to_datetime(ibit['Date'], errors='coerce')
if 'Price' in ibit.columns:
    ibit_price = (
        ibit['Price'].astype(str)
        .str.replace(',', '', regex=False)
        .str.replace('$', '', regex=False)
        .astype(float)
    )
elif 'IBIT_price' in ibit.columns:
    ibit_price = ibit['IBIT_price'].astype(float)
else:
    raise ValueError("IBIT CSV must contain either 'Price' or 'IBIT_price' column")

ibit_df = pd.DataFrame({'IBIT_close': ibit_price.values}, index=ibit['Date']).sort_index()
ibit_df = ibit_df[~ibit_df.index.duplicated(keep='last')].dropna(subset=['IBIT_close'])

# Exact FINAL CELL C IBIT mapping
ibit_combo_pos = best_combo_pos.reindex(ibit_df.index).ffill().fillna(0.0)
ibit_combo_pos = ibit_combo_pos.clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
ibit_combo_results = run_backtest_from_position(ibit_df['IBIT_close'], ibit_combo_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

# AMMA on IBIT
amma_df = amma_from_ibit_csv(
    str(crypto_root / 'Data' / 'IBIT_Data.csv'),
    momentum_weights={5:0.25, 10:0.25, 20:0.25, 60:0.25},
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
).rename(columns={'Date':'date'}).set_index('date')
amma_pos = amma_df.reindex(ibit_df.index)['amma_position'].fillna(0.0).clip(0.0, 1.0)
amma_results = run_backtest_from_position(ibit_df['IBIT_close'], amma_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

# Buy-hold
ibit_bh_pos = pd.Series(1.0, index=ibit_df.index)
ibit_bh_results = run_backtest_from_position(ibit_df['IBIT_close'], ibit_bh_pos, fee_bps=0.0, slippage_bps=0.0)

returns = pd.DataFrame({
    'BuyHold_IBIT': ibit_bh_results['net_ret'],
    'AMMA_IBIT': amma_results['net_ret'],
    'BestCombo_TMO_IBIT': ibit_combo_results['net_ret'],
}, index=ibit_df.index)

strategy_table = metrics_table(returns)
strategy_table


In [ ]:
# Strategy plots

eq = (1.0 + returns).cumprod()
dd = eq / eq.cummax() - 1.0

plt.figure(figsize=(12,5))
plt.plot(eq.index, eq['BuyHold_IBIT'], label='Buy & Hold', linewidth=2)
plt.plot(eq.index, eq['AMMA_IBIT'], label='AMMA', linewidth=2)
plt.plot(eq.index, eq['BestCombo_TMO_IBIT'], label=f"IBIT {best['label']}", linewidth=2)
plt.title('IBIT Equity Curves: Buy & Hold vs AMMA vs Exact Research-02 Combo')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(12,4))
for c in dd.columns:
    plt.plot(dd.index, dd[c], label=c)
plt.title('IBIT Drawdowns')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


In [ ]:
# 3) Portfolio scenarios A/B/C (existing assets proxy: SPY + AAPL)

def load_price_series(path: Path) -> pd.Series:
    x = pd.read_csv(path, encoding='utf-8-sig')
    x['Date'] = pd.to_datetime(x['Date'], format='%m/%d/%y', errors='coerce')
    px = pd.to_numeric(x['Price'].astype(str).str.replace(',', '', regex=False).str.replace('$', '', regex=False), errors='coerce')
    return pd.Series(px.values, index=x['Date']).dropna().sort_index()

spy = load_price_series(quant_root / 'Market Data' / 'Equity ETF' / 'SPY.csv')
aapl = load_price_series(quant_root / 'Market Data' / 'Equity ETF' / 'AAPL.csv')

base = pd.DataFrame({'SPY': spy.pct_change(), 'AAPL': aapl.pct_change()}).dropna()
all_ret = base.join(returns, how='inner').dropna()


def port_ret(df: pd.DataFrame, w: dict) -> pd.Series:
    ws = pd.Series(w, dtype=float)
    ws = ws / ws.sum()
    return df[ws.index].mul(ws, axis=1).sum(axis=1)

# A replace
A_base = port_ret(all_ret, {'SPY':0.4,'AAPL':0.4,'AMMA_IBIT':0.2})
A_new  = port_ret(all_ret, {'SPY':0.4,'AAPL':0.4,'BestCombo_TMO_IBIT':0.2})

# B add
B_without = A_base
B_with    = port_ret(all_ret, {'SPY':1/3,'AAPL':1/3,'AMMA_IBIT':1/6,'BestCombo_TMO_IBIT':1/6})

# C combine sleeves
C_equal = B_with
best_w, best_s = 0.5, -np.inf
for w in np.linspace(0,1,101):
    ibit_sleeve = w*all_ret['AMMA_IBIT'] + (1-w)*all_ret['BestCombo_TMO_IBIT']
    r = port_ret(pd.concat([all_ret[['SPY','AAPL']], ibit_sleeve.rename('IBIT_SLEEVE')], axis=1).dropna(), {'SPY':0.4,'AAPL':0.4,'IBIT_SLEEVE':0.2})
    s = sr(r)
    if np.isfinite(s) and s > best_s:
        best_s, best_w = s, float(w)
C_sharpe = port_ret(pd.concat([all_ret[['SPY','AAPL']], (best_w*all_ret['AMMA_IBIT']+(1-best_w)*all_ret['BestCombo_TMO_IBIT']).rename('IBIT_SLEEVE')], axis=1).dropna(), {'SPY':0.4,'AAPL':0.4,'IBIT_SLEEVE':0.2})

portfolio_tbl = metrics_table(pd.DataFrame({
    'A_Baseline_AMMA': A_base,
    'A_Replace_With_ExactCombo': A_new,
    'B_Add_ExactCombo': B_with,
    'C_Equal_AMMA_Combo': C_equal,
    'C_SharpeMax_AMMA_Combo': C_sharpe,
}))

corr_amma_combo = float(all_ret['AMMA_IBIT'].corr(all_ret['BestCombo_TMO_IBIT']))
print('Correlation(AMMA, ExactCombo)=', corr_amma_combo)
print('Best Sharpe-max AMMA weight inside IBIT sleeve (C):', best_w)
portfolio_tbl


In [ ]:
# 4) Clear quantitative conclusion

print('=== Strategy table ===')
print(strategy_table)
print('\n=== Portfolio table ===')
print(portfolio_tbl)

s_amma = strategy_table.loc['AMMA_IBIT', 'Sharpe']
s_combo = strategy_table.loc['BestCombo_TMO_IBIT', 'Sharpe']
d_amma = strategy_table.loc['AMMA_IBIT', 'MaxDD']
d_combo = strategy_table.loc['BestCombo_TMO_IBIT', 'MaxDD']

print('\n=== Decision ===')
if (s_combo > s_amma) and (d_combo >= d_amma):
    print('Trend+Mining+OU exact model improves risk-adjusted return vs AMMA and does not worsen drawdown materially -> replacement is justified.')
else:
    print('Trend+Mining+OU exact model does not dominate AMMA on Sharpe + drawdown -> keep AMMA or combine, not full replacement.')

best_port = portfolio_tbl.index[0]
print('Best portfolio scenario by Sharpe:', best_port)
if 'Add' in best_port or 'Combo' in best_port:
    print('Combining AMMA with the exact model adds diversification value in this run.')
else:
    print('Combination did not win; standalone allocation remains preferable in this run.')


In [ ]:
# 5) EXACT model printout requested (commented)

exact_model_comment = f"""
# ----------------------------
# FINAL CELL D: Equity curve on IBIT (combination model strategy)
# ----------------------------
# Selected combination model from Research 02 exact logic:
#   1) trend_mining_pos = (0.5 * trend_pos + 0.5 * mining_pos).clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
#      trend_mining_pos = trend_mining_pos.shift(1).fillna(0.0)
#   2) For a in np.linspace(0,1,21):
#         combo_pos = ((1-a)*trend_mining_pos + a*ou_pos_aligned).clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
#         combo_pos = combo_pos.shift(1).fillna(0.0)
#      keep best Sharpe -> best_combo_pos
#   3) Map to IBIT dates:
#         ibit_pos = best_combo_pos.reindex(ibit_df.index).ffill().fillna(0.0)
#         ibit_pos = ibit_pos.clip(-cfg.LEVERAGE_CAP, cfg.LEVERAGE_CAP)
#   4) Backtest + plot:
#         plt.plot(ibit_combo_results['net_equity'], label=f"IBIT {{best['label']}}")
#         plt.plot(ibit_bh_results['net_equity'], label='IBIT Buy & Hold', linestyle='--')
#
# Current selected model instance:
#   label={best['label']}
#   alpha_ou={best['alpha_ou']}
#   Sharpe={best['sr']}
# ----------------------------
"""
print(exact_model_comment)
